In [3]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange

pd.set_option("display.max_columns", None)

In [4]:
dataset = requests.get('https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&limit=108048').json()
resale_df = pd.DataFrame(dataset['result']['records'])

In [5]:
rows, cols = resale_df.shape
print('Number of rows: {}'.format(rows))
print('Number of columns : {}'.format(cols))

Number of rows: 108048
Number of columns : 12


In [6]:
resale_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108048 entries, 0 to 108047
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   town                 108048 non-null  object
 1   flat_type            108048 non-null  object
 2   flat_model           108048 non-null  object
 3   floor_area_sqm       108048 non-null  object
 4   street_name          108048 non-null  object
 5   resale_price         108048 non-null  object
 6   month                108048 non-null  object
 7   remaining_lease      108048 non-null  object
 8   lease_commence_date  108048 non-null  object
 9   storey_range         108048 non-null  object
 10  _id                  108048 non-null  int64 
 11  block                108048 non-null  object
dtypes: int64(1), object(11)
memory usage: 9.9+ MB


In [7]:
resale_df.head()

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,_id,block
0,SERANGOON,EXECUTIVE,Maisonette,152,SERANGOON NTH AVE 4,720000,2017-02,74 years 09 months,1992,04 TO 06,1,526
1,SERANGOON,EXECUTIVE,Maisonette,152,SERANGOON NTH AVE 4,725000,2017-02,74 years 09 months,1992,10 TO 12,2,525
2,SERANGOON,EXECUTIVE,Maisonette,154,SERANGOON CTRL,938888,2017-02,70 years 11 months,1989,10 TO 12,3,423
3,TAMPINES,3 ROOM,Simplified,64,TAMPINES ST 83,290000,2017-02,69 years 09 months,1987,01 TO 03,4,849
4,TAMPINES,3 ROOM,Simplified,64,TAMPINES ST 44,292000,2017-02,69 years 05 months,1987,10 TO 12,5,462


In [11]:
def onemap_geocoding(address):
    result = requests.get('https://developers.onemap.sg/commonapi/search?searchVal='+address+'&returnGeom=Y&getAddrDetails=Y&pageNum=1')
    result = eval(result.text)
    
    if result['found'] > 0:
        lat, long = result['results'][0]['LATITUDE'], result['results'][0]['LONGITUDE']
        return address, lat, long
    else:
        pass
def geocode_series(addresses):
    coord_list = []
    failed_list = []
    failed = 0
    t = trange(len(addresses), desc='Geoding Addresses')
    for i in t:
        try:
            result = onemap_geocoding(addresses[i])
            if len(result) > 0:
                coord_list.append(result)
                t.set_description('{} failed'.format(failed))
        except:
            failed_list.append(addresses[i])
            failed += 1
            t.set_description('{} failed'.format(failed))
    
    return coord_list
resale_df['address'] = resale_df['block'] + ' ' + resale_df['street_name']
house_addresses = resale_df['address'].unique()



In [ ]:
pip install tensoflow